# Dynamics 365 Business Central Trouble Shooting Guide (TSG) - Extensions

This notebook contains Kusto queries that can help getting to the root cause of an issue with extensions for one or more environments. 

NB! Some of the signal used in this notebook is only available in newer versions of Business Central, so check the version of your environment if some sections do not return any data. The signal documentation states in which version a given signal was introduced.

## 1. Connect to Application Insights
First you need to set the notebook Kernel to Python3, load the KQLmagic module (did you install it?) and connect to your Application Insights resource (get appid and appkey from the API access page in the Application Insights portal)

In [20]:
# load the KQLmagic module
%reload_ext Kqlmagic

In [21]:
# Connect to the Application Insights API
%kql appinsights://appid='<add app id from the Application Insights portal>';appkey='<add API key from the Application Insights portal>'


## 2. Define filters
This workbook is designed for troubleshooting extensions. Please provide values for aadTenantId, environmentName, and extensionId.

You can also specify limits to the period of time that the analysis should include.

In [22]:
# Add values for AAD tenant id, environment name, and extension id here
#
# It is possible to leave one or more values blank (if you want to analyze across all values of the parameter)
# 
aadTenantId = ""
environmentName = ""
extensionId = ""

# date filters for the analysis
# use YYYY-MM-DD format for the dates (ISO 8601)
startDate = "2020-09-20"
endDate = "2020-09-27"

# Analyze extension events
Now you can run Kusto queries to look for possible root causes for issues about extensions.

Either click **Run All** above to run all sections, or scroll down to the type of analysis you want to do and manually run queries

## Extension event overview

Event telemetry docs: 
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-extension-lifecycle-trace
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-extension-update-trace

KQL samples: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/ExtensionLifecycle.kql

In [23]:
%%kql
//
// extension event types stats 
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _extensionId = extensionId;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)
    and (_aadTenantId == '' or customDimensions.aadTenantId == _aadTenantId)
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and (_extensionId == '' or customDimensions.extensionId == _extensionId)    
    and customDimensions.eventId in ('RT0010', 'LC0010', 'LC0011', 'LC0012', 'LC0013', 'LC0014', 'LC0015', 'LC0016', 'LC0017', 'LC0018', 'LC0019', 'LC020', 'LC021', 'LC022', 'LC023')    
| extend aadTenantId=tostring( customDimensions.aadTenantId)
       , environmentName=tostring( customDimensions.environmentName )
       , extensionId=tostring( customDimensions.extensionId )
       , eventId=tostring(customDimensions.eventId)
| extend eventMessageShort=case(
              eventId=='RT0010', strcat(eventId, ': Extension Update Failed (exception raised in upgrade code)')
            , eventId=='LC0010', strcat(eventId, ': Extension installed successfully')
            , eventId=='LC0011', strcat(eventId, ': Extension failed to install')
            , eventId=='LC0012', strcat(eventId, ': Extension synchronized successfully')            
            , eventId=='LC0013', strcat(eventId, ': Extension failed to synchronize')            
            , eventId=='LC0014', strcat(eventId, ': Extension published successfully')            
            , eventId=='LC0015', strcat(eventId, ': Extension failed to publish')            
            , eventId=='LC0016', strcat(eventId, ': Extension un-installed successfully')            
            , eventId=='LC0017', strcat(eventId, ': Extension failed to un-install')            
            , eventId=='LC0018', strcat(eventId, ': Extension unpublished successfully')
            , eventId=='LC0019', strcat(eventId, ': Extension failed to un-publish')
            , eventId=='LC0020', strcat(eventId, ': Extension compiled successfully')
            , eventId=='LC0021', strcat(eventId, ': Extension failed to compile')
            , eventId=='LC0022', strcat(eventId, ': Extension updated successfully')
            , eventId=='LC0023', strcat(eventId, ': Extension failed to update')
            , strcat( eventId, ': Unknown message')
         )
| summarize count() by eventMessageShort
| order by eventMessageShort
| render barchart with (title='Extension lifecycle event overview', legend=hidden)

[{'eventMessageShort': 'LC0018: Extension unpublished successfully', 'count_': 27},
 {'eventMessageShort': 'LC0016: Extension un-installed successfully', 'count_': 28},
 {'eventMessageShort': 'LC0015: Extension failed to publish', 'count_': 1},
 {'eventMessageShort': 'LC0014: Extension published successfully', 'count_': 33},
 {'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 59},
 {'eventMessageShort': 'LC0012: Extension synchronized successfully', 'count_': 696},
 {'eventMessageShort': 'LC0010: Extension installed successfully', 'count_': 34}]

In [24]:
%%kql
//
// top 100 extension events
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _extensionId = extensionId;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)
    and (_aadTenantId == '' or customDimensions.aadTenantId == _aadTenantId)
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and (_extensionId == '' or customDimensions.extensionId == _extensionId)    
    and customDimensions.eventId in ('RT0010', 'LC0010', 'LC0011', 'LC0012', 'LC0013', 'LC0014', 'LC0015', 'LC0016', 'LC0017', 'LC0018', 'LC0019', 'LC020', 'LC021', 'LC022', 'LC023')    
| extend aadTenantId=tostring( customDimensions.aadTenantId)
       , environmentName=tostring( customDimensions.environmentName )
       , extensionId=tostring( customDimensions.extensionId )
       , extensionName=tostring( customDimensions.extensionName )
| limit 100
| project aadTenantId, environmentName, extensionId, extensionName, timestamp, message
| order by aadTenantId asc, environmentName asc, extensionId asc, timestamp desc

,aadTenantId,environmentName,extensionId,extensionName,timestamp,message
0,0845cf51-2012-445e-8a44-fdd0df28584b,Sandbox,437dbf0e-84ff-417a-965d-ed2bb9650972,Base Application,2020-09-22 16:14:11.568779300+00:00,Extension synchronized successfully: Base Appl...
1,0952b452-7ca5-43a8-b4cb-9f020d64a567,Sandbox,437dbf0e-84ff-417a-965d-ed2bb9650972,Base Application,2020-09-22 16:14:41.267285400+00:00,Extension synchronized successfully: Base Appl...
2,1377ae6d-ed0c-45bd-a6cc-473f83d13e03,Production,437dbf0e-84ff-417a-965d-ed2bb9650972,Base Application,2020-09-26 21:58:26.660655900+00:00,Extension synchronized successfully: Base Appl...
3,148eff18-320d-4c99-95ac-6b1c9739195f,Sandbox,437dbf0e-84ff-417a-965d-ed2bb9650972,Base Application,2020-09-23 23:43:51.868805+00:00,Extension synchronized successfully: Base Appl...
4,14f6e7c1-10ef-4d9c-a5b7-87592cf8d066,Sandbox,437dbf0e-84ff-417a-965d-ed2bb9650972,Base Application,2020-09-27 03:16:33.846260500+00:00,Extension synchronized successfully: Base Appl...
5,14f6e7c1-10ef-4d9c-a5b7-87592cf8d066,Sandbox,8afe7b40-8c87-4beb-ada0-451d1761bf95,_Exclude_APIV1_,2020-09-27 03:16:59.869288800+00:00,Extension synchronized successfully: _Exclude_...
6,1525097a-c341-473b-b235-3def2adb15de,Production,437dbf0e-84ff-417a-965d-ed2bb9650972,Base Application,2020-09-26 21:58:29.587821+00:00,Extension synchronized successfully: Base Appl...
7,1525097a-c341-473b-b235-3def2adb15de,Production,437dbf0e-84ff-417a-965d-ed2bb9650972,Base Application,2020-09-23 21:10:28.828653100+00:00,Extension synchronized successfully: Base Appl...
8,187ad3be-63c6-48da-b603-a00799aa63bc,Production,437dbf0e-84ff-417a-965d-ed2bb9650972,Base Application,2020-09-24 00:07:12.250764700+00:00,Extension synchronized successfully: Base Appl...
9,18a38716-7d7e-49b4-a1fe-faa1cd2b30c6,Sandbox,437dbf0e-84ff-417a-965d-ed2bb9650972,Base Application,2020-09-22 16:13:35.839006100+00:00,Extension synchronized successfully: Base Appl...


[{'aadTenantId': '0845cf51-2012-445e-8a44-fdd0df28584b', 'environmentName': 'Sandbox', 'extensionId': '437dbf0e-84ff-417a-965d-ed2bb9650972', 'extensionName': 'Base Application', 'timestamp': datetime.datetime(2020, 9, 22, 16, 14, 11, 568779, tzinfo=tzutc()), 'message': 'Extension synchronized successfully: Base Application version 16.5.15897.16808 by Microsoft (437dbf0e-84ff-417a-965d-ed2bb9650972)'},
 {'aadTenantId': '0952b452-7ca5-43a8-b4cb-9f020d64a567', 'environmentName': 'Sandbox', 'extensionId': '437dbf0e-84ff-417a-965d-ed2bb9650972', 'extensionName': 'Base Application', 'timestamp': datetime.datetime(2020, 9, 22, 16, 14, 41, 267285, tzinfo=tzutc()), 'message': 'Extension synchronized successfully: Base Application version 16.5.15897.16808 by Microsoft (437dbf0e-84ff-417a-965d-ed2bb9650972)'},
 {'aadTenantId': '1377ae6d-ed0c-45bd-a6cc-473f83d13e03', 'environmentName': 'Production', 'extensionId': '437dbf0e-84ff-417a-965d-ed2bb9650972', 'extensionName': 'Base Application', 'timestamp': datetime.datetime(2020, 9, 26, 21, 58, 26, 660655, tzinfo=tzutc()), 'message': 'Extension synchronized successfully: Base Application version 16.5.15897.17062 by Microsoft (437dbf0e-84ff-417a-965d-ed2bb9650972)'},
 {'aadTenantId': '148eff18-320d-4c99-95ac-6b1c9739195f', 'environmentName': 'Sandbox', 'extensionId': '437dbf0e-84ff-417a-965d-ed2bb9650972', 'extensionName': 'Base Application', 'timestamp': datetime.datetime(2020, 9, 23, 23, 43, 51, 868805, tzinfo=tzutc()), 'message': 'Extension synchronized successfully: Base Application version 16.5.15897.16869 by Microsoft (437dbf0e-84ff-417a-965d-ed2bb9650972)'},
 {'aadTenantId': '14f6e7c1-10ef-4d9c-a5b7-87592cf8d066', 'environmentName': 'Sandbox', 'extensionId': '437dbf0e-84ff-417a-965d-ed2bb9650972', 'extensionName': 'Base Application', 'timestamp': datetime.datetime(2020, 9, 27, 3, 16, 33, 846260, tzinfo=tzutc()), 'message': 'Extension synchronized successfully: Base Application version 16.5.15897.16869 by Microsoft (437dbf0e-84ff-417a-965d-ed2bb9650972)'},
 {'aadTenantId': '14f6e7c1-10ef-4d9c-a5b7-87592cf8d066', 'environmentName': 'Sandbox', 'extensionId': '8afe7b40-8c87-4beb-ada0-451d1761bf95', 'extensionName': '_Exclude_APIV1_', 'timestamp': datetime.datetime(2020, 9, 27, 3, 16, 59, 869288, tzinfo=tzutc()), 'message': 'Extension synchronized successfully: _Exclude_APIV1_ version 16.5.15897.16869 by Microsoft (8afe7b40-8c87-4beb-ada0-451d1761bf95)'},
 {'aadTenantId': '1525097a-c341-473b-b235-3def2adb15de', 'environmentName': 'Production', 'extensionId': '437dbf0e-84ff-417a-965d-ed2bb9650972', 'extensionName': 'Base Application', 'timestamp': datetime.datetime(2020, 9, 26, 21, 58, 29, 587821, tzinfo=tzutc()), 'message': 'Extension synchronized successfully: Base Application version 16.5.15897.17062 by Microsoft (437dbf0e-84ff-417a-965d-ed2bb9650972)'},
 {'aadTenantId': '1525097a-c341-473b-b235-3def2adb15de', 'environmentName': 'Production', 'extensionId': '437dbf0e-84ff-417a-965d-ed2bb9650972', 'extensionName': 'Base Application', 'timestamp': datetime.datetime(2020, 9, 23, 21, 10, 28, 828653, tzinfo=tzutc()), 'message': 'Extension synchronized successfully: Base Application version 16.5.15897.16982 by Microsoft (437dbf0e-84ff-417a-965d-ed2bb9650972)'},
 {'aadTenantId': '187ad3be-63c6-48da-b603-a00799aa63bc', 'environmentName': 'Production', 'extensionId': '437dbf0e-84ff-417a-965d-ed2bb9650972', 'extensionName': 'Base Application', 'timestamp': datetime.datetime(2020, 9, 24, 0, 7, 12, 250764, tzinfo=tzutc()), 'message': 'Extension synchronized successfully: Base Application version 16.5.15897.16869 by Microsoft (437dbf0e-84ff-417a-965d-ed2bb9650972)'},
 {'aadTenantId': '18a38716-7d7e-49b4-a1fe-faa1cd2b30c6', 'environmentName': 'Sandbox', 'extensionId': '437dbf0e-84ff-417a-965d-ed2bb9650972', 'extensionName': 'Base Application', 'timestamp': datetime.datetime(2020, 9, 22, 16, 13, 35, 839006, tzinfo=tzutc()), 'message': 'Extension synchronized successfully: Base Application version 16.5.1

## Extension failures

Event telemetry docs: 
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-extension-lifecycle-trace
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-extension-update-trace

In [25]:
%%kql
//
// extension event failure overview
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _extensionId = extensionId;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)
    and (_aadTenantId == '' or customDimensions.aadTenantId == _aadTenantId)
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and (_extensionId == '' or customDimensions.extensionId == _extensionId)    
    and customDimensions.eventId in ('RT0010', 'LC0011', 'LC0013', 'LC0015', 'LC0017', 'LC0019', 'LC021', 'LC023')    
| extend aadTenantId=tostring( customDimensions.aadTenantId)
       , environmentName=tostring( customDimensions.environmentName )
       , extensionId=tostring( customDimensions.extensionId )
       , eventId=tostring(customDimensions.eventId)
| extend eventMessageShort=case(
              eventId=='RT0010', strcat(eventId, ': Extension Update Failed (exception raised in upgrade code)')
            , eventId=='LC0011', strcat(eventId, ': Extension failed to install')
            , eventId=='LC0013', strcat(eventId, ': Extension failed to synchronize')            
            , eventId=='LC0015', strcat(eventId, ': Extension failed to publish')            
            , eventId=='LC0017', strcat(eventId, ': Extension failed to un-install')            
            , eventId=='LC0019', strcat(eventId, ': Extension failed to un-publish')
            , eventId=='LC0021', strcat(eventId, ': Extension failed to compile')
            , eventId=='LC0023', strcat(eventId, ': Extension failed to update')
            , strcat( eventId, ': Unknown message')
         )
| summarize count() by eventMessageShort
| order by eventMessageShort
| render columnchart with (title='Failure type overview', xtitle="", legend=hidden)

[{'eventMessageShort': 'LC0015: Extension failed to publish', 'count_': 1},
 {'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 59}]

In [26]:
%%kql
//
// top 100 extension event failure details
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _extensionId = extensionId;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)
    and (_aadTenantId == '' or customDimensions.aadTenantId == _aadTenantId)
    and (_environmentName == '' or customDimensions.environmentName == _environmentName )
    and (_extensionId == '' or customDimensions.extensionId == _extensionId)
    and customDimensions.eventId in ('RT0010', 'LC0011', 'LC0013', 'LC0015', 'LC0017', 'LC0019', 'LC021', 'LC023')    
| extend aadTenantId=tostring( customDimensions.aadTenantId)
       , environmentName=tostring( customDimensions.environmentName )
       , extensionId=tostring( customDimensions.extensionId )
       , eventId=tostring(customDimensions.eventId)
       , extensionName=tostring(customDimensions.extensionName)
| extend eventMessageShort=case(
              eventId=='RT0010', strcat(eventId, ': Extension Update Failed (exception raised in upgrade code)')
            , eventId=='LC0011', strcat(eventId, ': Extension failed to install')
            , eventId=='LC0013', strcat(eventId, ': Extension failed to synchronize')            
            , eventId=='LC0015', strcat(eventId, ': Extension failed to publish')            
            , eventId=='LC0017', strcat(eventId, ': Extension failed to un-install')            
            , eventId=='LC0019', strcat(eventId, ': Extension failed to un-publish')
            , eventId=='LC0021', strcat(eventId, ': Extension failed to compile')
            , eventId=='LC0023', strcat(eventId, ': Extension failed to update')
            , strcat( eventId, ': Unknown message')
         )
| summarize count() by aadTenantId, environmentName, extensionId, extensionName, eventMessageShort
| order by count_ desc, aadTenantId asc, environmentName asc, extensionId asc, extensionName asc, eventMessageShort asc
| limit 100

,aadTenantId,environmentName,extensionId,extensionName,eventMessageShort,count_
0,3aaabf66-9e80-4dc0-bbb4-9f94a456a847,Production,,VAT Group Management,LC0013: Extension failed to synchronize,1
1,4f7334eb-dc1d-4bc2-8423-50ad417e53af,Sandbox,,Application,LC0013: Extension failed to synchronize,1
2,4f7334eb-dc1d-4bc2-8423-50ad417e53af,Sandbox,,Base Application,LC0013: Extension failed to synchronize,1
3,4f7334eb-dc1d-4bc2-8423-50ad417e53af,Sandbox,,Business Central Cloud Migration - Previous Re...,LC0013: Extension failed to synchronize,1
4,4f7334eb-dc1d-4bc2-8423-50ad417e53af,Sandbox,,Business Central Intelligent Cloud,LC0013: Extension failed to synchronize,1
5,4f7334eb-dc1d-4bc2-8423-50ad417e53af,Sandbox,,Essential Business Headlines,LC0013: Extension failed to synchronize,1
6,4f7334eb-dc1d-4bc2-8423-50ad417e53af,Sandbox,,Intelligent Cloud Base,LC0013: Extension failed to synchronize,1
7,4f7334eb-dc1d-4bc2-8423-50ad417e53af,Sandbox,,Late Payment Prediction,LC0013: Extension failed to synchronize,1
8,4f7334eb-dc1d-4bc2-8423-50ad417e53af,Sandbox,,PayPal Payments Standard,LC0013: Extension failed to synchronize,1
9,4f7334eb-dc1d-4bc2-8423-50ad417e53af,Sandbox,,Sales and Inventory Forecast,LC0013: Extension failed to synchronize,1


[{'aadTenantId': '3aaabf66-9e80-4dc0-bbb4-9f94a456a847', 'environmentName': 'Production', 'extensionId': '', 'extensionName': 'VAT Group Management', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extensionId': '', 'extensionName': 'Application', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extensionId': '', 'extensionName': 'Base Application', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extensionId': '', 'extensionName': 'Business Central Cloud Migration - Previous Release', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extensionId': '', 'extensionName': 'Business Central Intelligent Cloud', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extensionId': '', 'extensionName': 'Essential Business Headlines', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extensionId': '', 'extensionName': 'Intelligent Cloud Base', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extensionId': '', 'extensionName': 'Late Payment Prediction', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extensionId': '', 'extensionName': 'PayPal Payments Standard', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extensionId': '', 'extensionName': 'Sales and Inventory Forecast', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extensionId': '', 'extensionName': 'Send To Email Printer', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extensionId': '', 'extensionName': 'Send remittance advice by email', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extensionId': '', 'extensionName': 'System Application', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extensionId': '', 'extensionName': 'VAT Group Management', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extensionId': '', 'extensionName': '_Exclude_APIV1_', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extensionId': '', 'extensionName': '_Exclude_AnonymizedDataSharing_', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extensionId': '', 'extensionName': '_Exclude_ClientAddIns_', 'eventMessageShort': 'LC0013: Extension failed to synchronize', 'count_': 1},
 {'aadTenantId': '4f7334eb-dc1d-4bc2-8423-50ad417e53af', 'environmentName': 'Sandbox', 'extension